## 📦 Passo 1: Importar Bibliotecas

**Novidade:** `QdrantVectorStore` e `QdrantClient`

In [1]:
import os
import requests
from pathlib import Path
from dotenv import load_dotenv

# LangChain core
from langchain_ollama import OllamaEmbeddings, OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

# Qdrant
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

# LCEL (Chains)
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Display
from IPython.display import Markdown, display
import pandas as pd

## ⚙️ Passo 2: Configurar Ambiente

**Lê variáveis de ambiente do `.env`:**
- `QDRANT_URL`: Endereço do Qdrant (padrão: `http://localhost:6333`)
- `QDRANT_API_KEY`: Chave de autenticação (opcional em dev)
- `OLLAMA_BASE_URL`: Endereço do Ollama

**Dica:** Se estiver rodando no Docker Compose, use os hostnames internos (`http://qdrant:6333`).

In [3]:
# Carrega variáveis de ambiente
load_dotenv()

# Configurações do Qdrant
QDRANT_URL = 'http://localhost:6333'
QDRANT_API_KEY = os.getenv('QDRANT_API_KEY')  # None se não configurado
COLLECTION_NAME = os.getenv('QDRANT_COLLECTION_NAME', 'rag_documents')

# Configurações do Ollama
OLLAMA_BASE_URL = 'http://localhost:11434'
EMBEDDING_MODEL = 'embeddinggemma'
LLM_MODEL = 'llama3.2:1b'

# Paths
BASE_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
PDF_DIR = BASE_DIR.parent.parent / "data" / "pdfs"

# Parâmetros de chunking
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
K = 4  # top-k documentos retornados

print(f"📁 Diretório de PDFs: {PDF_DIR}")
print(f"🗄️  Qdrant URL: {QDRANT_URL}")
print(f"🖼️  Dashboard URL {QDRANT_URL}/dashboard")
print(f"📦 Collection: {COLLECTION_NAME}")
print(f"🤖 Ollama URL: {OLLAMA_BASE_URL}")
print(f"🔢 Parâmetros: chunk_size={CHUNK_SIZE}, k={K}")

📁 Diretório de PDFs: e:\01-projetos\11-work\11.34-engenharia-vetorial\data\pdfs
🗄️  Qdrant URL: http://localhost:6333
🖼️  Dashboard URL http://localhost:6333/dashboard
📦 Collection: documents_collection
🤖 Ollama URL: http://localhost:11434
🔢 Parâmetros: chunk_size=1000, k=4


## 🔍 Passo 3: Verificar Conexão com Qdrant

Checagem simples para garantir que o Qdrant está acessível.

In [4]:
# Testa conexão com Qdrant
print('Verificando Qdrant...')
resp = requests.get(f"{QDRANT_URL.replace('http://qdrant', 'http://localhost')}/")
print(f'Qdrant status: {resp.status_code} ({resp.reason})')
print(f'Versão: {resp.json().get("version", "N/A")}')

# Testa Ollama
print('\nVerificando Ollama...')
resp_ollama = requests.post(f"{OLLAMA_BASE_URL.replace('http://ollama', 'http://localhost')}/api/show")
print(f'Ollama status: {resp_ollama.status_code}')

Verificando Qdrant...
Qdrant status: 200 (OK)
Versão: 1.16.1

Verificando Ollama...
Ollama status: 400


## 📄 Passo 4: Carregar Documentos

Mesmo processo do lab anterior: carrega PDFs e cria lista de documentos.

In [5]:
# Carrega documentos
documents = []
pdf_paths = list(PDF_DIR.glob("*.pdf"))
print(f"📚 Encontrados {len(pdf_paths)} PDFs\n")

for pdf_path in pdf_paths:
    loader = PyPDFLoader(str(pdf_path))
    docs = loader.load()
    documents.extend(docs)
    print(f"  ✓ {pdf_path.name}: {len(docs)} páginas")

print(f"\n✅ Total de páginas carregadas: {len(documents)}")

incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams
incorrect startxref pointer(1)
parsing for Object Streams


📚 Encontrados 4 PDFs

  ✓ api_documentation.pdf: 3 páginas
  ✓ livro_receitas.pdf: 5 páginas
  ✓ manual_futebol.pdf: 4 páginas
  ✓ manual_iphone15.pdf: 3 páginas

✅ Total de páginas carregadas: 15


## ✂️ Passo 5: Dividir em Chunks

Mesmo processo: divide documentos em chunks menores.

In [6]:
# Chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

chunks = text_splitter.split_documents(documents=documents)

print(f"✂️  Total de chunks criados: {len(chunks)}")
print(f"📊 Expansão de chunking: {len(chunks)/len(documents):.1f}x")

✂️  Total de chunks criados: 33
📊 Expansão de chunking: 2.2x


## 🧮 Passo 6: Criar Embeddings

Usamos o Ollama para gerar embeddings (mesmo do lab anterior).

In [8]:
# Cria embeddings
embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,
    base_url=OLLAMA_BASE_URL
)

print(f"🧠 Modelo de embeddings: {EMBEDDING_MODEL}")
print("✅ Embeddings configurados!")

# Testa com um texto de exemplo
test_embedding = embeddings.embed_query("teste")
print(f"📐 Dimensões do embedding: {len(test_embedding)}")

🧠 Modelo de embeddings: embeddinggemma
✅ Embeddings configurados!
📐 Dimensões do embedding: 768


## 🗄️ Passo 7: Conectar ao Qdrant e Criar Collection

### O que é uma Collection?

No Qdrant, uma **collection** é como uma "tabela" em bancos SQL. Ela armazena:
- **Vetores** (embeddings)
- **Payloads** (metadados em JSON: source, page, etc.)
- **IDs** únicos para cada documento

### Configuração da Collection

- **`vectors_config`**: Define dimensão e métrica de distância
- **`Distance.COSINE`**: Usa similaridade cosseno (mesma do FAISS)
- **Persistência automática**: Dados salvos em `/qdrant/storage` (volume Docker)

### Idempotência

O código verifica se a collection já existe. Se sim, **reutiliza** (não recria).

In [9]:
# Conecta ao Qdrant
qdrant_client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
    timeout=60
)

print(f"🔌 Conectado ao Qdrant: {QDRANT_URL}")

# Lista collections existentes
collections = qdrant_client.get_collections()
existing_names = [c.name for c in collections.collections]
print(f"📦 Collections existentes: {existing_names}")

# Cria collection se não existir
if COLLECTION_NAME not in existing_names:
    print(f"\n🆕 Criando collection '{COLLECTION_NAME}'...")
    qdrant_client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(
            size=len(test_embedding),  # Dimensão do embedding
            distance=Distance.COSINE    # Métrica: cosseno
        )
    )
    print("✅ Collection criada!")
else:
    print(f"\n♻️  Reutilizando collection existente '{COLLECTION_NAME}'")

# Info da collection
collection_info = qdrant_client.get_collection(COLLECTION_NAME)
print(f"\n📊 Collection Info:")
print(f"   Vetores: {collection_info.points_count}")
print(f"   Dimensões: {collection_info.config.params.vectors.size}")
print(f"   Distância: {collection_info.config.params.vectors.distance}")

🔌 Conectado ao Qdrant: http://localhost:6333
📦 Collections existentes: ['test_collection', 'tech_docs']

🆕 Criando collection 'documents_collection'...
✅ Collection criada!

📊 Collection Info:
   Vetores: 0
   Dimensões: 768
   Distância: Cosine


C:\Users\Max\AppData\Local\Temp\ipykernel_29228\1376114093.py:2: UserWarning: Api key is used with an insecure connection.
  qdrant_client = QdrantClient(


## 📥 Passo 8: Indexar Documentos no Qdrant

### Como funciona?

O `QdrantVectorStore.from_documents()` faz automaticamente:
1. Gera embeddings para cada chunk
2. Cria um ID único para cada documento
3. Armazena embedding + metadata (payload) no Qdrant
4. **Persiste no disco** (volume Docker)

### Persistência

Diferente do FAISS (que precisa de `save_local()`), o Qdrant **persiste automaticamente**. Os dados ficam em:
- Container: `/qdrant/storage`
- Host: Volume Docker `qdrant_storage`

**Vantagem:** Ao reiniciar o container, os dados permanecem!

In [11]:
# Indexa documentos no Qdrant
print("⏳ Indexando documentos no Qdrant...")
print("   (Isso pode demorar alguns minutos na primeira vez)\n")

vectorstore = QdrantVectorStore.from_documents(
    documents=chunks,
    embedding=embeddings,
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
    collection_name=COLLECTION_NAME,
    force_recreate=False  # False = reutiliza se existir
)

print("✅ Documentos indexados!")

# Estatísticas pós-indexação
collection_info = qdrant_client.get_collection(COLLECTION_NAME)
print(f"\n📊 Collection atualizada:")
print(f"   Total de vetores: {collection_info.points_count}")
print(f"   Status: {collection_info.status}")

⏳ Indexando documentos no Qdrant...
   (Isso pode demorar alguns minutos na primeira vez)

✅ Documentos indexados!

📊 Collection atualizada:
   Total de vetores: 66
   Status: green


## 🔍 Passo 9: Criar Retriever

O retriever do Qdrant funciona igual ao do FAISS, mas com mais opções:
- **`search_type="similarity"`**: Busca por similaridade cosseno
- **`search_type="mmr"`**: Maximum Marginal Relevance (diversidade)
- **`search_kwargs`**: Configurações extras (k, score_threshold, etc.)

In [13]:
# Cria retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": K}
)

print(f"🔍 Retriever criado!")
print(f"📋 Configuração: top-{K} chunks mais similares")

🔍 Retriever criado!
📋 Configuração: top-4 chunks mais similares


## 💬 Passo 10: Criar Prompt e LLM

Mesmos componentes do lab anterior.

In [14]:
# Template de prompt
template = """Use o seguinte contexto para responder à pergunta.
Se você não souber a resposta baseado no contexto, diga "Não encontrei essa informação nos documentos fornecidos."

Importante: Cite sempre a fonte (nome do arquivo e página) quando possível.

Contexto:
{context}

Pergunta: {question}

Resposta detalhada:"""

prompt = PromptTemplate(
    template=template,
    input_variables=['context', 'question']
)

# Cria LLM
llm = OllamaLLM(
    model=LLM_MODEL,
    base_url=OLLAMA_BASE_URL.replace('http://ollama', 'http://localhost')
)

print("📝 Prompt template criado!")
print(f"🤖 LLM configurado: {LLM_MODEL}")

📝 Prompt template criado!
🤖 LLM configurado: llama3.2:1b


## 🔗 Passo 11: Criar Chain RAG com LCEL

Mesma chain do lab anterior, mas agora usando o retriever do Qdrant!

In [15]:
# Helper para formatar documentos
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Cria a Chain RAG
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("🔗 Chain RAG criada com sucesso!")
print("✅ Sistema pronto para uso!")

🔗 Chain RAG criada com sucesso!
✅ Sistema pronto para uso!


## 🚀 Passo 12: Testar o Sistema RAG

Vamos fazer uma pergunta e ver a resposta!

In [16]:
# Pergunta de teste
query = "Quais são as principais formações táticas do futebol?"

print(f"❓ Pergunta: {query}\n")
print("⏳ Processando...\n")
print("=" * 80)

# Invoca a chain
response = rag_chain.invoke(query)

print("\n🤖 Resposta:")
print("=" * 80)
print(response)
print("=" * 80)

❓ Pergunta: Quais são as principais formações táticas do futebol?

⏳ Processando...


🤖 Resposta:
As principais formações táticas utilizadas no futebol podem ser classificadas em duas categorias principais: a Formação Equilibrada Clássica e a Formação Ofensiva com Wingers.

A Formação Equilibrada Clássica é caracterizada por uma estrutura defensiva robusta, onde dois zagueiros centrais defendem o centro do campo, seguidos por quatro zagueiros laterais que mantêm boa cobertura defensiva. Além disso, há dois meias campistas e duas alas (direitas e esquerdas) para jogar pelo meio-campo. Essa formação é conhecida como 4-4-2.

A Formação Ofensiva com Wingers é caracterizada por uma estrutura de centroavante, onde um atacante central flancia pelo meio do campo enquanto dois wingers laterais seguem em direção ao adversário. Além disso, há três meias campistas e um volante para jogar pelo meio-campo. Essa formação é conhecida como 4-3-3.

No entanto, é importante notar que existem variantes de

## 🔍 Passo 13: Inspecionar Documentos Recuperados

Vamos ver quais chunks o Qdrant retornou.

In [17]:
# Busca manual para inspeção
relevant_docs = retriever.invoke(query)

print(f"📚 Documentos recuperados: {len(relevant_docs)}\n")

# Exibe em formato tabular
rows = []
for i, doc in enumerate(relevant_docs, 1):
    rows.append({
        'doc': i,
        'source': doc.metadata.get('source', 'N/A'),
        'page': doc.metadata.get('page', 'N/A'),
        'snippet': doc.page_content[:200].replace('\n', ' ')
    })

df = pd.DataFrame(rows)
display(df)

📚 Documentos recuperados: 4



,doc,source,page,snippet
0,1,e:\01-projetos\11-work\11.34-engenharia-vetori...,2,Formações Táticas Clássicas 4-4-2 (Formação Eq...
1,2,e:\01-projetos\11-work\11.34-engenharia-vetori...,2,Formações Táticas Clássicas 4-4-2 (Formação Eq...
2,3,e:\01-projetos\11-work\11.34-engenharia-vetori...,3, Mantém organização espacial do time  Dificu...
3,4,e:\01-projetos\11-work\11.34-engenharia-vetori...,3, Mantém organização espacial do time  Dificu...


## 🌊 Passo 14: Streaming (Bonus)

Resposta em tempo real, palavra por palavra!

In [18]:
# Teste com streaming
query_stream = "Como fazer uma lasanha?"

print(f"❓ Pergunta: {query_stream}\n")
print("🌊 Resposta (streaming):")
print("=" * 80)

for chunk in rag_chain.stream(query_stream):
    print(chunk, end="", flush=True)

print("\n" + "=" * 80)
print("\n✅ Streaming concluído!")

❓ Pergunta: Como fazer uma lasanha?

🌊 Resposta (streaming):
Para preparar uma lasanha, você precisará seguir os passos abaixo:

1. **Preparar o molho branco**: Em uma panela média, derreta a manteiga em fogo médio-baixo, adicione a farinha de trigo e mexa vigorosamente por 2 minutos para formar um roux (pasta dourada). Adicione o leite aos poucos, mexendo constantemente, até engrossar e começar a ferver (8-10 minutos).

   Para essa parte, é importante lembrar de não economizar no molho branco. Continue mexendo até que ele esteja bem cozido.

2. **Preparar o bechamel**: Em outra panela, derreta a manteiga em fogo baixo e adicione a farinha de trigo, mexendo vigorosamente por 1 minuto para formar uma crosta (pasta dourada). Adicione a leite aos poucos, mexendo constantemente, até engrossar. Continue mexendo por mais 2 minutos.

3. **Preparar as conchas de molho**: Espalhe 2 conchas de molho bolonhesa no fundo de um refratário grande (35x25cm).

4. **Montagem da lasanha**:
   - Pré-aque

---

## 📊 Estatísticas do Sistema

Vamos ver estatísticas completas do nosso sistema RAG com Qdrant.

In [19]:
# Estatísticas
print("📊 Estatísticas do Sistema RAG com Qdrant\n")
print("=" * 80)

# Dados
print(f"\n📚 Dados:")
print(f"   Total de PDFs: {len(pdf_paths)}")
print(f"   Total de páginas: {len(documents)}")
print(f"   Total de chunks: {len(chunks)}")
print(f"   Expansão de chunking: {len(chunks)/len(documents):.1f}x")

# Chunks
tamanhos = [len(c.page_content) for c in chunks]
print(f"\n📏 Chunks:")
print(f"   Tamanho médio: {sum(tamanhos)/len(tamanhos):.0f} caracteres")
print(f"   Tamanho mínimo: {min(tamanhos)} caracteres")
print(f"   Tamanho máximo: {max(tamanhos)} caracteres")

# Qdrant collection
collection_info = qdrant_client.get_collection(COLLECTION_NAME)
print(f"\n🗄️  Qdrant Collection '{COLLECTION_NAME}':")
print(f"   Total de vetores: {collection_info.points_count}")
print(f"   Dimensões: {collection_info.config.params.vectors.size}")
print(f"   Distância: {collection_info.config.params.vectors.distance}")
print(f"   Status: {collection_info.status}")

# Chain
print(f"\n🔗 Chain RAG:")
print(f"   Componentes: 5 (retriever → format → prompt → llm → parser)")
print(f"   Suporte a streaming: ✅ Sim")
print(f"   Persistência: ✅ Automática (Qdrant)")

print("\n" + "=" * 80)

📊 Estatísticas do Sistema RAG com Qdrant


📚 Dados:
   Total de PDFs: 4
   Total de páginas: 15
   Total de chunks: 33
   Expansão de chunking: 2.2x

📏 Chunks:
   Tamanho médio: 815 caracteres
   Tamanho mínimo: 262 caracteres
   Tamanho máximo: 997 caracteres

🗄️  Qdrant Collection 'documents_collection':
   Total de vetores: 66
   Dimensões: 768
   Distância: Cosine
   Status: green

🔗 Chain RAG:
   Componentes: 5 (retriever → format → prompt → llm → parser)
   Suporte a streaming: ✅ Sim
   Persistência: ✅ Automática (Qdrant)

